# Train_net

In [2]:
import os

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, build_detection_train_loader, DatasetCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, verify_results
from detectron2.projects.point_rend import add_pointrend_config
from detectron2.utils.logger import setup_logger

from detectron2.projects.lidar_sup import LidarSupDatasetMapper, add_point_sup_config


In [12]:
    cfg = get_cfg()
    add_pointrend_config(cfg)
    add_point_sup_config(cfg)
    cfg.merge_from_file('configs/mask_rcnn_R_50_FPN_3x_point_sup_coco.yaml')
    cfg.DATASETS.TRAIN = 'nuscenes'
    cfg.DATASETS.TEST = 'nuscenes'
    cfg.freeze()

In [16]:
cfg['MODEL']['ROI_HEADS']

CfgNode({'NAME': 'StandardROIHeads', 'NUM_CLASSES': 80, 'IN_FEATURES': ['p2', 'p3', 'p4', 'p5'], 'IOU_THRESHOLDS': [0.5], 'IOU_LABELS': [0, 1], 'BATCH_SIZE_PER_IMAGE': 512, 'POSITIVE_FRACTION': 0.25, 'SCORE_THRESH_TEST': 0.05, 'NMS_THRESH_TEST': 0.5, 'PROPOSAL_APPEND_GT': True})

In [17]:
cfg['SOLVER']

CfgNode({'LR_SCHEDULER_NAME': 'WarmupMultiStepLR', 'MAX_ITER': 1200, 'BASE_LR': 0.02, 'MOMENTUM': 0.9, 'NESTEROV': False, 'WEIGHT_DECAY': 0.0001, 'WEIGHT_DECAY_NORM': 0.0, 'GAMMA': 0.1, 'STEPS': (800, 1000), 'WARMUP_FACTOR': 0.001, 'WARMUP_ITERS': 1000, 'WARMUP_METHOD': 'linear', 'CHECKPOINT_PERIOD': 5000, 'IMS_PER_BATCH': 2, 'REFERENCE_WORLD_SIZE': 0, 'BIAS_LR_FACTOR': 1.0, 'WEIGHT_DECAY_BIAS': 0.0001, 'CLIP_GRADIENTS': CfgNode({'ENABLED': False, 'CLIP_TYPE': 'value', 'CLIP_VALUE': 1.0, 'NORM_TYPE': 2.0}), 'AMP': CfgNode({'ENABLED': False})})

In [6]:
trainer = Trainer(cfg)

[07/09 16:14:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[07/09 16:14:52 d2.projects.lidar_sup.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[07/09 16:14:52 d2.projects.lidar_sup.dataset_mapper]: Point Augmentations used in training: sample 0 points
WARNING [07/09 16:14:52 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/09 16:14:52 d2.data.datasets.coco]: Loaded 2424 images in COCO format from /home/PJLAB/lixiang/Desktop/weaklysup/nuscenes_2dbox_10points_without_mask.json
[07/09 16:14:52 d2.data.build]: Removed 679 images with no usable annotations. 1745 images left.
[07/09 16:14:52 d2.data.build]: Distribution of instances among all 10 categories:
|  category  | #instances   |   category    | #instances   |   category   | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:------------:|:-------------|
|    c

In [9]:
data = next(trainer._trainer._data_loader_iter)

In [4]:
class Trainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
        if evaluator_type == "coco":
            evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        elif len(evaluator_list) == 1:
            return evaluator_list[0]
        return DatasetEvaluators(evaluator_list)
    @classmethod
    def build_train_loader(cls, cfg):
        if cfg.INPUT.POINT_SUP:
            mapper = LidarSupDatasetMapper(cfg, is_train=True)
        else:
            mapper = None
        return build_detection_train_loader(cfg, mapper=mapper)

# Registery

In [12]:
from detectron2.utils.registry import Registry

In [13]:
META_ARCH_REGISTRY = Registry("META_ARCH")

In [19]:
@META_ARCH_REGISTRY.register()
def print_hello_world(word):
    print('hello {}'.format(word))

In [21]:
META_ARCH_REGISTRY.get('print_hello_world')('hello registery')

hello hello registery


In [20]:
META_ARCH_REGISTRY

Registry of META_ARCH:
╒═══════════════════╤════════════════════════════════════════════════╕
│ Names             │ Objects                                        │
╞═══════════════════╪════════════════════════════════════════════════╡
│ print_hello_world │ <function print_hello_world at 0x7f7e76578d30> │
╘═══════════════════╧════════════════════════════════════════════════╛

In [71]:
from detectron2.data.build import get_detection_dataset_dicts

In [72]:
dataset = get_detection_dataset_dicts(
            cfg.DATASETS.TRAIN,
            filter_empty=cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS,
            min_keypoints=cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE
            if cfg.MODEL.KEYPOINT_ON
            else 0,
            proposal_files=cfg.DATASETS.PROPOSAL_FILES_TRAIN if cfg.MODEL.LOAD_PROPOSALS else None,
        )

WARNING [07/09 15:14:03 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/09 15:14:03 d2.data.datasets.coco]: Loaded 2424 images in COCO format from /home/PJLAB/lixiang/Desktop/weaklysup/nuscenes_2dbox_10points_without_mask.json
[07/09 15:14:03 d2.data.build]: Removed 679 images with no usable annotations. 1745 images left.


In [73]:
len(dataset)

1745

In [75]:
num_pic_with_no_ann = 0
for i, data in enumerate(dataset):
    if len(data['annotations']) == 0:
        num_pic_with_no_ann += 1
print(num_pic_with_no_ann)

0


In [7]:
import numpy as np
[1 for _ in range(10)]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]